### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
import datetime


%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
movie_list = []
for movie in list(movies['genre'].unique()):
    if type(movie) != float:
        movie_list.extend(movie.split('|'))
    
len(set(movie_list))

28

True

In [31]:
list(movies['genre'].unique())[1] == np.nan

False

In [10]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [43]:
reviews['rating'].min()

0

In [ ]:
# and even more - open additional cells as necessary

In [48]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': movies.shape[0],
'The number of ratings in the dataset': reviews.shape[0],
'The number of different genres': len(set(movie_list)),
'The number of unique users in the dataset': len(reviews['user_id'].unique()),
'The number missing ratings in the reviews dataset': reviews['rating'].isnull().sum(),
'The average rating given across all ratings': round(reviews['rating'].mean(),0),
'The minimum rating given across all ratings': reviews['rating'].min(),
'The maximum rating given across all ratings': reviews['rating'].max()
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


In [49]:
t.q1_check(dict_sol1)

That looks good to me!


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [2]:
movies['date'] = movies['movie'].apply(lambda x: x[-5:-1] if x[-1] == ")" else np.nan)
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [3]:
x = movies.set_index('movie').genre.str.split(r'|', expand=True).stack().reset_index(level=1, drop=True).to_frame('genre')
x_genre_dummies = pd.get_dummies(x['genre']).groupby(level=0).sum()

In [4]:
movies['century'] = movies['date'].apply(lambda x: x[:2]+"00's")
movies.head()

,movie_id,movie,genre,date,century
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1800's
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1800's
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1800's
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1800's
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1800's


In [5]:
x_cent_dummies = pd.get_dummies(movies['century'])

In [6]:
movies = movies.merge(x_genre_dummies, left_on='movie', right_on='movie')

In [7]:
movies_clean = pd.concat([movies.drop('century', axis = 1), x_cent_dummies], axis = 1)
movies_clean.head()

,movie_id,movie,genre,date,Action,Adult,Adventure,Animation,Biography,Comedy,...,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0
3,0000091,Le manoir du diable (1896),Short|Horror,1896,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0
4,0000131,Une nuit terrible (1896),Short|Comedy|Horror,1896,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0,0


In [8]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [9]:
reviews['date'] = reviews['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)))
reviews['month'] = reviews['date'].astype('datetime64[ns]').dt.month
reviews['year'] = reviews['date'].astype('datetime64[ns]').dt.year
reviews['month'] = reviews['month'].astype('str')
reviews['year'] = reviews['year'].astype('str')

In [10]:
x_month_dummies = pd.get_dummies(reviews['month'], prefix ='month')
x_year_dummies = pd.get_dummies(reviews['year'], prefix = 'year')

reviews_clean = pd.concat([reviews[['user_id', 'movie_id', 'rating', 'timestamp', 'date']], x_month_dummies, x_year_dummies], axis = 1)
reviews_clean.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_10,month_11,month_12,month_2,...,month_8,month_9,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,1,0114508,8,1381006850,2013-10-05 21:00:50,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,0208092,5,1586466072,2020-04-09 21:01:12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0358273,9,1579057827,2020-01-15 03:10:27,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year

In [15]:
movies_new.drop('Unnamed: 0', axis = 1).sort_index(inplace=True) == movies_clean.sort_index(inplace=True)

True

In [19]:
reviews_new.drop('Unnamed: 0', axis = 1).sort_index(inplace=True) == reviews_clean.sort_index(inplace=True)

True